# Gerador dos arquivos de entrada

In [ ]:
def gerar_arquivo(tamanho_bytes, nome_arquivo):
    conteudo = "seguranca-heitor"
    tamanho_string = len(conteudo)
    repeticoes = tamanho_bytes // tamanho_string
    restante = tamanho_bytes % tamanho_string

    with open(nome_arquivo, 'w') as f:
        f.write(conteudo * repeticoes)
        if restante > 0:
            f.write(conteudo[:restante])


gerar_arquivo(1 * 1024 * 1024, 'arquivo_1MB.txt')

gerar_arquivo(100 * 1024 * 1024, 'arquivo_100MB.txt')

gerar_arquivo(1 * 1024 * 1024 * 1024, 'arquivo_1GB.txt')

# Gerador do csv

```sh

#!/bin/bash

k128_hex='FE384EF6E548B5A0F4E1133D91E11CEE'
k192_hex='C4A7E2A786AC2C4930B3B0CABD947D4D3E530C24A9C2B59'
k256_hex='8C790DBF3D13E4DE19900C15C3598A6B2F2C70E47373410818A981AA3A4B831'
iv_hex='A16FB13930D3C6F81884DBBB4881E3B2'

files=("arquivo_1MB.txt" "arquivo_100MB.txt" "arquivo_1GB.txt")
file_sizes=("1MB" "100MB" "1GB")
key_sizes=("128" "192" "256")
keys=($k128 $k192 $k256)

cipher_file() {
    local file=$1
    local key_hex=$2
    local iv_hex=$3
    local key_size=$4
    local cipher_file="ciphered_${key_size}_${file}"
    { time openssl enc -aes-${key_size}-cbc -K "$key_hex" -iv "$iv_hex" -in "$file" -out "$cipher_file"; } 2>&1 | grep real | awk '{print $2}'
}

echo "file_size,key_size,time" > resultados.csv

for j in {0..2}; do
    file="${files[$j]}"
    file_size="${file_sizes[$j]}"
    for i in {0..2}; do
        key_size=${key_sizes[$i]}
        key_hex=${keys[$i]}
        
        # Cifrando o arquivo 10 vezes e gravando os tempos no CSV
        for iteracao in {1..10}; do
            tempo=$(cipher_file "$file" "$key_hex" "$iv_hex" "$key_size")
            # Convertendo o tempo do formato mm:ss para segundos
            minutos=$(echo $tempo | cut -d'm' -f1)
            segundos=$(echo $tempo | cut -d'm' -f2 | cut -d's' -f1)
            tempo_total=$(echo "$minutos*60 + $segundos" | bc)
            
            echo "$file_size,$key_size,$tempo_total" >> resultados.csv
        done
    done
done
```

In [9]:
import pandas as pd
df = pd.read_csv(r'resultados.csv')

In [10]:
pd.set_option('display.max_rows', None)
print(df)

   file_size  key_size    time
0        1MB       128   0.169
1        1MB       128   0.012
2        1MB       128   0.007
3        1MB       128   0.006
4        1MB       128   0.005
5        1MB       128   0.007
6        1MB       128   0.006
7        1MB       128   0.006
8        1MB       128   0.006
9        1MB       128   0.005
10       1MB       192   0.006
11       1MB       192   0.007
12       1MB       192   0.006
13       1MB       192   0.005
14       1MB       192   0.006
15       1MB       192   0.007
16       1MB       192   0.006
17       1MB       192   0.006
18       1MB       192   0.006
19       1MB       192   0.007
20       1MB       256   0.006
21       1MB       256   0.008
22       1MB       256   0.006
23       1MB       256   0.006
24       1MB       256   0.007
25       1MB       256   0.007
26       1MB       256   0.006
27       1MB       256   0.006
28       1MB       256   0.008
29       1MB       256   0.007
30     100MB       128   0.938
31     1

In [11]:
df.groupby(['key_size', 'file_size'])[
    'time'].agg(['mean', 'std']).reset_index()

,key_size,file_size,mean,std
0,128,100MB,1.4672,0.297102
1,128,1GB,17.5592,4.874239
2,128,1MB,0.0229,0.051373
3,192,100MB,1.5406,0.832106
4,192,1GB,18.1167,3.082050
5,192,1MB,0.0062,0.000632
6,256,100MB,1.4379,0.983982
7,256,1GB,17.9891,6.536529
8,256,1MB,0.0067,0.000823
